# Wasserstein distance estimation

In [1]:
import sys
sys.path.append("/home/lauro/code/msc-thesis/svgd/kernel_learning")
sys.path.append("/home/lauro/code/msc-thesis/wassdistance")

import jax
import jax.numpy as np
from jax import grad, jit, vmap, random, lax, jacfwd
from jax import lax
from jax.ops import index_update, index
from jax.experimental import optimizers

import matplotlib.pyplot as plt
import numpy as onp
from tqdm import tqdm
import time
from functools import partial

import utils
import metrics
import plot
from svgd import SVGD
import svgd
import stein
import kernels
import train

rkey = random.PRNGKey(0)

from jax.config import config; config.update("jax_log_compiles", 1)

/home/lauro/.virtualenvs/msc-thesis/lib/python3.8/site-packages/jax/lib/xla_bridge.py:125: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


# Get samples

In [35]:
d1 = distributions.Gaussian([0, 0], 1)
d2 = distributions.Gaussian([0, 0.5], 1)

n = 5000
m = 5000
sa = d1.sample(n)
d2.newkey()
sb = d2.sample(m)
a = np.ones(n) / n
b = np.ones(m) / m

distance = "sqeuclidean"
# distance = "euclidean"

from scipy.spatial.distance import cdist
C = cdist(sa, sb, distance)

In [36]:
ot.bregman.empirical_sinkhorn_divergence(sa, sb, 1, metric=distance)

array([0.2939886])

## Python Optimal Transport

In [25]:
import ot
emd = ot.emd2(a, b, C, numItermax=10**6)
if distance == "sqeuclidean":
    print("EMD =", np.sqrt(emd))
else:
    print("EMD =", emd)

EMD = 0.10683221


## Scipy

Implementation idea from here: https://stackoverflow.com/questions/57562613/python-earth-mover-distance-of-2d-arrays

NOTE: only works if both sets of samples have the same number of samples. That's the premise behind using linear_sum_assignment here.

In [26]:
from scipy.optimize import linear_sum_assignment
assignment = linear_sum_assignment(C)
emd = C[assignment].sum() / n

if distance == "sqeuclidean":
    print("EMD =", np.sqrt(emd))
else:
    print("EMD =", emd)

EMD = 0.10683221


## Analytical Wasserstein distance between two gaussians

Let $\mu_{1}=\mathcal{N}\left(m_{1}, C_{1}\right)$ and $\mu_{2}=\mathcal{N}\left(m_{2}, C_{2}\right)$ be two non-degenerate Gaussian measures (i.e. normal distributions) on $\mathbb{R}^{n},$ with respective expected values $m_{1}$ and $m_{2} \in \mathbb{R}^{n}$ and symmetric positive semi-definite covariance matrices $C_{1}$ and $C_{2} \in \mathbb{R}^{n \times n} .$ Then, with respect to the usual Euclidean norm on $\mathbb{R}^{n},$ the 2
Wasserstein distance between $\mu_{1}$ and $\mu_{2}$ is

$$W_{2}\left(\mu_{1}, \mu_{2}\right)^{2}=\left\|m_{1}-m_{2}\right\|_{2}^{2}+\operatorname{trace}\left(C_{1}+C_{2}-2\left(C_{2}^{1 / 2} C_{1} C_{2}^{1 / 2}\right)^{1 / 2}\right)$$

In [27]:
def analytical_wasserstein(ma, mb, cova, covb):
    """Compute Wasserstein distance of order 2 between two Gaussians."""
    ma = np.asarray(ma)
    mb = np.asarray(mb)
    cova = np.asarray(cova)
    covb = np.asarray(covb)
    sqrtcovb = np.sqrt(covb)
    if np.squeeze(ma).ndim > 0:
        wsquared = np.sum((ma - mb)**2) + np.trace(cova + covb - 2 * np.sqrt(np.matmul(np.matmul(sqrtcovb, cova), sqrtcovb)))
        return np.sqrt(wsquared)
    else:
        return np.sqrt((ma - mb)**2 + cova + covb - 2 * np.sqrt(cova * covb))

In [28]:
analytical_wasserstein(d1.mean, d2.mean, d1.cov, d2.cov)

DeviceArray(0., dtype=float32)

# Sinkhorn distance

In [29]:
sinkhorn = ot.sinkhorn2(a, b, C, 1, numItermax=10**6)
sinkhorn

array([0.88222351])

In [30]:
ot.bregman.empirical_sinkhorn2(sa, sb, 1, metric=distance)

array([0.88222353])

In [31]:
ot.bregman.empirical_sinkhorn2(sa, sb, 1, metric=distance)

array([0.88222353])

In [ ]:
ot.bregman.empirical_sinkhorn_divergence(sa, sb, 1, metric=distance)